# Extremely Minimal, Dirty Bleak Scanner implementation 

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio
import time

In [3]:
from bleak import BleakClient, BleakScanner
from bleak.backends.device import BLEDevice

In [4]:
USE_PYCYCLING_EXAMPLE=False
if USE_PYCYCLING_EXAMPLE:
    from pycycling.sterzo import Sterzo

In [5]:
HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"

In [6]:
class BleakFSMError(Exception):
    pass

class StaleScanError(Exception):
    pass

In [7]:
class BleakModel:
    '''
    This class is a transitions.AsyncModel wrapper around the BleakScanner class.
    It is used to scan for Bluetooth Low Energy devices and return the results within the state machine framework.

    Since most of the methods are meant to be state machine callbacks, 
    they are prefixed with an underscore to prevent accidentally calling them directly.

    '''
    bt_devices = {} # class variable to store the discovered devices, since we can only have one BleakScanner
    _stop_scan_event = asyncio.Event() # class variable to stop the scan
    def __init__(self, scan_stale_time=120, auto_rescan=True, scan_timeout=3):
        '''
        Args
            (int) scan_stale_time: number of seconds while scan results are considered valid for establishing connection. Must re-scan if connection attempted after this time.
            (bool) auto_rescan: If connect() attempted when scan is stale, run scan for `scan_timeout` seconds
            (int) scan_timeout: Number of seconds to scan, if auto_rescan is True
        '''
        self.bleak_client: BleakClient = None
        self.ble_device: BLEDevice = None
        self.target = None
        self._stop_streaming_event = asyncio.Event()

        self.scan_stale_time = scan_stale_time
        self.auto_rescan = auto_rescan
        self.scan_timeout = scan_timeout

        self.last_scan_time = None

        if USE_PYCYCLING_EXAMPLE:
            self.sterzo: Sterzo = None

    def set_target_address(self, address):
        if address in BleakModel.bt_devices:
            self.target = address
            return True
        else:
            return BleakFSMError(f"Address {address} not found in discovered devices")

    async def _bt_scan(self):        
        def detection_callback(device, _advertisement_data):
            BleakModel.bt_devices[device.address] = device
        async with BleakScanner(detection_callback) as scanner:
            await self._stop_scan_event.wait() # continues to scan until stop_scan_event is set


    async def _stop_bt_scan(self):
        self.last_scan_time = time.time()

        self._stop_scan_event.set()

    async def _connect_to_device(self):
        # stale scan data
        if self.last_scan_time is not None:
            if time.time() - self.last_scan_time > self.scan_stale_time:
                if self.auto_rescan:
                    print("Stale scan. Automatically re-scanning before attempting connect")
                    await self._bt_scan()
                    await asyncio.sleep(self.scan_timeout)
                    await self._stop_bt_scan()
                    await self._connect_to_device() # while this looks recursive, it isn't because after one iteration, the stale time will not have been reached
                    return True
                else:
                    raise StaleScanError(f"It's been over {self.scan_stale_time} seconds since last scan. Scan first (model.start_scan()), and Reconnect (model.connect()). Or, set `auto_rescan=True` to do this automatically")
                
        if len(BleakModel.bt_devices) == 0:
            raise BleakFSMError("No devices found")
            # return False
        try:
            self.ble_device = (BleakModel.bt_devices.pop(self.target))# remove the device from the list to avoid connecting to it multiple times
        except:
            raise BleakFSMError(f"Bluetooth device {self.target} not found in scanned list. It could be powered off, connected to another device, or to another BleakModel.")
        self.bleak_client = BleakClient(self.ble_device) # we don't use the async context manager because we want to access the client object from the disconnect function
        try:
            connected = await self.bleak_client.connect()
            if connected:
                print(f"Connected to {self.target}")

                # If we want to create a class abstraction for the bluetooth device,
                # Instantiate it here and pass in the bleak_client. For example:
                if USE_PYCYCLING_EXAMPLE:
                    self.sterzo = Sterzo(self.bleak_client)
                else:
                    pass # Otherwise, we can just use the bleak_client itself later on.


                return True
            else:
                print(f"Failed to connect to {self.target}")
        except Exception as e:
            print(f"An error occurred: {e}")
            return False
        return True
    
    async def _disconnect_from_device(self):
        BleakModel.bt_devices[self.target] = self.ble_device # put it back in the list
        await self.bleak_client.disconnect()
        print(f"Disconnected from {self.bleak_client.address}")

    async def _stream_from_device(self):
        self._stop_streaming_event.clear()

        if USE_PYCYCLING_EXAMPLE:
            def steering_handler(steering_angle):
                print(steering_angle)
            
            self.sterzo.set_steering_measurement_callback(steering_handler)
            await self.sterzo.enable_steering_measurement_notifications()
        else: # use heart rate example
            def handle_hr_measurement(sender, data):
                print(f"Data received from {sender}: {data}")
                flag = data[0]
                heart_rate = data[1]
                print(f"Heart Rate: {heart_rate} beats per minute")
            
            await self.bleak_client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)

        print("Started streaming")

        await self._stop_streaming_event.wait()
        return True

    async def _stop_stream_from_device(self):
        self._stop_streaming_event.set()
        try:
            if USE_PYCYCLING_EXAMPLE:
                await self.sterzo.disable_steering_measurement_notifications()
            else:
                await self.bleak_client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
                
            print("Stopped streaming")
            return True
        except Exception as e:
            print(f"An error occurred while stopping streaming: {e}")
            return False

    async def _stop_stream_and_disconnect_from_device(self):
        '''
        Since we can't re-use the connection after stopping notify,
        we bundle the stop streaming and disconnect logic
        so that the state jumps from Streaming to Init.
        '''
        await self._stop_stream_from_device()
        await self._disconnect_from_device()
        return True
    

In [8]:

model = BleakModel(
    auto_rescan=True
)

transitions = []

machine = AsyncMachine(model, states=["Init", "Scanning", "Connected", "Streaming"], transitions=transitions, initial='Init')

machine.add_transition(
    trigger="start_scan",
    source="Init",
    dest="Scanning",
    after="_bt_scan"
)

machine.add_transition(
    trigger="stop_scan",
    source="Scanning",
    dest="Init",
    after="_stop_bt_scan"
)

machine.add_transition(
    trigger="connect",
    source="Init",
    dest="Connected",
    conditions="_connect_to_device",
)

machine.add_transition(
    trigger="stream",
    source="Connected",
    dest="Streaming",
    after="_stream_from_device"
)

# The following two "disconnect" triggers are for
# Connected -> Init (which is obvious) and
# Streaming -> Init: After a stream is stopped, we can't go back to Connected
# because we can't re-use the BleakClient object.
# Therefore we need to go one more back to Init,
# then to Connect, and then back to Streaming.

machine.add_transition(
    trigger="disconnect",
    source="Connected",
    dest="Init",
    before="_disconnect_from_device"
)


machine.add_transition(
    trigger="disconnect",
    source="Streaming",
    dest="Init",
    before="_stop_stream_and_disconnect_from_device"
)

In [9]:
# If we try to run connect() without having scanned, it should fail 
# and raise BleakFSMError
# specifically, `is_device_list_empty` will return True, and the `unless` condition will fail

try:
    await model.connect()
except BleakFSMError as e:
    print(f"Correctly raised error: {e}")

Correctly raised error: No devices found


In [10]:
# Since we're in a Jupyter notebook, 
# we need to use `create_task()` instead of `get_event_loop().run_until_complete()`
# to use the existing event loop which Jupyter runs on.
asyncio.create_task(model.start_scan())
# if we used
# `await model.start_scan()`
# then it would work, but it would block until the scan is finished, which is not what we want,
# because below we will show how the `model.bt_devices` dictionary gets filled with the results of the scan.

<Task pending name='Task-7' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [11]:
model.state

'Scanning'

In [12]:
async def stop_after_delay(delay, model):
    print("Starting sleep for 2 seconds (giving the scanner time to scan)")
    await asyncio.sleep(delay)
    print("Slept for 2 seconds (sending end signal to scanner)")
    await model.stop_scan()
    print("Stopped scanning")
    

asyncio.create_task(stop_after_delay(2, model))

<Task pending name='Task-9' coro=<stop_after_delay() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_49872/921128545.py:1>>

In [13]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

Starting sleep for 2 seconds (giving the scanner time to scan)
0 devices found in 0.0 seconds.
0 devices found in 0.05 seconds.
0 devices found in 0.1 seconds.
0 devices found in 0.15 seconds.
2 devices found in 0.2 seconds.
6 devices found in 0.25 seconds.
6 devices found in 0.3 seconds.
6 devices found in 0.35 seconds.
7 devices found in 0.4 seconds.
7 devices found in 0.45 seconds.
7 devices found in 0.5 seconds.
7 devices found in 0.55 seconds.
7 devices found in 0.6 seconds.
7 devices found in 0.65 seconds.
7 devices found in 0.7 seconds.
7 devices found in 0.75 seconds.
7 devices found in 0.8 seconds.
7 devices found in 0.85 seconds.
7 devices found in 0.9 seconds.
7 devices found in 0.95 seconds.


In [14]:
model.bt_devices

{'14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
 'CAFED98A-0110-DC79-35AD-4E015094E64B': BLEDevice(CAFED98A-0110-DC79-35AD-4E015094E64B, None),
 '99FF17C7-A82C-5DD9-195F-6F4E1810746E': BLEDevice(99FF17C7-A82C-5DD9-195F-6F4E1810746E, None),
 'FE3F10AF-3FA6-0F72-1532-09BC89AF7C43': BLEDevice(FE3F10AF-3FA6-0F72-1532-09BC89AF7C43, None),
 '39613AEE-762C-928A-2BA6-9965101AF3C2': BLEDevice(39613AEE-762C-928A-2BA6-9965101AF3C2, None),
 '3984D4FE-75B5-36B2-CBC9-A4A24710D75D': BLEDevice(3984D4FE-75B5-36B2-CBC9-A4A24710D75D, None),
 '45D8FE73-595E-3847-80B4-37CAD9A697E0': BLEDevice(45D8FE73-595E-3847-80B4-37CAD9A697E0, None),
 '46AA3748-C6A6-8B21-4E79-8B1F0D8A95B4': BLEDevice(46AA3748-C6A6-8B21-4E79-8B1F0D8A95B4, None)}

In [15]:
await asyncio.sleep(3)

Slept for 2 seconds (sending end signal to scanner)
Stopped scanning


In [16]:
# Client should select the ID from the following list

for k,v in model.bt_devices.items():
    print(v)

14863C4D-BF71-4EA3-C6B4-98001056AAF8: WHOOPDEDOO
CAFED98A-0110-DC79-35AD-4E015094E64B: None
99FF17C7-A82C-5DD9-195F-6F4E1810746E: None
FE3F10AF-3FA6-0F72-1532-09BC89AF7C43: None
39613AEE-762C-928A-2BA6-9965101AF3C2: None
3984D4FE-75B5-36B2-CBC9-A4A24710D75D: None
45D8FE73-595E-3847-80B4-37CAD9A697E0: None
46AA3748-C6A6-8B21-4E79-8B1F0D8A95B4: None
91C2295A-4DE3-ED34-1544-0F7BAECEE4F3: None


In [17]:
# Client should set the target address 

model.set_target_address("14863C4D-BF71-4EA3-C6B4-98001056AAF8")

True

In [18]:
await model.connect() # first time connecting

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [19]:
model.state

'Connected'

In [20]:
# a second bluetooth model

model2 = BleakModel()
machine.add_model(model2)

In [21]:
await model2.start_scan()
asyncio.sleep(3)
await model2.stop_scan()

/var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_49872/1274855101.py:2: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(3)


True

In [22]:
model2.bt_devices

{'CAFED98A-0110-DC79-35AD-4E015094E64B': BLEDevice(CAFED98A-0110-DC79-35AD-4E015094E64B, None),
 '99FF17C7-A82C-5DD9-195F-6F4E1810746E': BLEDevice(99FF17C7-A82C-5DD9-195F-6F4E1810746E, None),
 'FE3F10AF-3FA6-0F72-1532-09BC89AF7C43': BLEDevice(FE3F10AF-3FA6-0F72-1532-09BC89AF7C43, None),
 '39613AEE-762C-928A-2BA6-9965101AF3C2': BLEDevice(39613AEE-762C-928A-2BA6-9965101AF3C2, None),
 '3984D4FE-75B5-36B2-CBC9-A4A24710D75D': BLEDevice(3984D4FE-75B5-36B2-CBC9-A4A24710D75D, None),
 '45D8FE73-595E-3847-80B4-37CAD9A697E0': BLEDevice(45D8FE73-595E-3847-80B4-37CAD9A697E0, None),
 '46AA3748-C6A6-8B21-4E79-8B1F0D8A95B4': BLEDevice(46AA3748-C6A6-8B21-4E79-8B1F0D8A95B4, None),
 '91C2295A-4DE3-ED34-1544-0F7BAECEE4F3': BLEDevice(91C2295A-4DE3-ED34-1544-0F7BAECEE4F3, None)}

In [24]:
model2.state

'Init'

In [25]:
model.state

'Connected'

In [26]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-20' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_49872/3697076873.py:1>>

Started streaming
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10D\xeb\x04')
Heart Rate: 68 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10D\xcc\x025\x02')
Heart Rate: 68 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10D|\x03')
Heart Rate: 68 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10DL\x03')
Heart Rate: 68 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10D)\x03')
Heart Rate: 68 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10C_\x03')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 

In [28]:
await asyncio.sleep(10)

Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10C}\x05')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10C\xee\x02')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x10C\xfe\x02')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00C')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00C')
Heart Rate: 67 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00B')
Heart Rate: 66 beats per minute
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray

In [29]:
await model.disconnect()

Stopped streaming
Disconnected from 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [30]:
model.state

'Init'